In [1]:
import pandas as pd
df1=pd.read_csv("Lisboa por IQR2.csv", index_col="id")
df=df1.drop(labels="Unnamed: 0", axis=1)
df
for col in df.columns:
    if df[col].nunique() == 2:
        print(f"{col}: {df[col].unique()}")

source: ['city scrape' 'previous scrape']
host_is_superhost: ['f' 't']
has_availability: ['t' 'f']
instant_bookable: ['t' 'f']


In [111]:
%%writefile lis.py

import streamlit as st
import plotly.express as px
import pandas as pd
import toml
from PIL import Image
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns


@st.cache_resource

def cargar_datos():

    
    df1=pd.read_csv("Lisboa por IQR2.csv")
    df=df1.drop(labels="Unnamed: 0", axis=1)

    dataframe_numerico = df.select_dtypes(['float','int'])
    columnas_numericas = dataframe_numerico.columns

    dataframe_texto = df.select_dtypes(["object"])
    columnas_texto = dataframe_texto.columns
 
    columnas_deseadas = {"source", "host_response_time", "host_is_superhost", "neighbourhood_group_cleansed", "neighbourhood_cleansed", "room_type", "instant_bookable", "property_type", "has_availability"}
    columnas_objetivo = [col for col in columnas_deseadas if col in df.columns]
    columnas_logistica = ['host_is_superhost','source', 'has_availability', 'instant_bookable', 'neighbourhood_group_cleansed', 'room_type', 'reviews_per_month', 'review_scores_rating']


    columna_categorica_fuente=df['source']
    columna_categorica_tiempo_de_respuesta_host=df['host_response_time']
    columna_categorica_host_es_superhost=df['host_is_superhost']
    #columna_categorica_tipo_habitacion=df['room_type']
    columna_categorica_reservacion_instantanea=df['instant_bookable']

    categorias_unicas_fuente = columna_categorica_fuente.unique()
    categorias_unicas_tiempo_de_respuesta_host = columna_categorica_tiempo_de_respuesta_host.unique()
    categorias_unicas_host_es_superhost = columna_categorica_host_es_superhost.unique()
    #categorias_unicas_tipo_habitacion = columna_categorica_tipo_habitacion.unique()
    categorias_unicas_reservacion_instantanea = columna_categorica_reservacion_instantanea.unique()

    config = toml.load("C:/Users/lalob/Documents/IS/Dashboard/.streamlit/config.toml")
    colores = ["#50e261", "#44c45c", "#62e276", "#3aad56", "#5cf298", "#72e261", "#34a44e", "#6ff2b4", "#a3e261", "#61e290", "#50e2a4", "#d0f2e2", "#2f9e63", "#77f294", "#50a1e2", "#e2f2d0", "#61e2c9", "#cccccc", "#f2d0e2", "#e2a450"]
    colores2 = ["#50e261", "#1fb4e7", "#f28f1a", "#a44fe2", "#f2c24d", "#3b9ae1", "#e2487e", "#8cf250", "#d94d41", "#5c3be2", "#50c0e2", "#e2b750", "#ff6347", "#228be6", "#d95091", "#50e2d0", "#6f42c1", "#ffcc00", "#17a2b8", "#e75f50"]

    return df, colores, colores2, columnas_logistica, columnas_numericas, columnas_texto, dataframe_numerico, dataframe_texto, categorias_unicas_fuente, categorias_unicas_tiempo_de_respuesta_host, categorias_unicas_host_es_superhost, categorias_unicas_reservacion_instantanea, columnas_objetivo

df, colores, colores2, columnas_logistica, columnas_numericas, columnas_texto, dataframe_numerico, dataframe_texto, categorias_unicas_fuente, categorias_unicas_tiempo_de_respuesta_host, categorias_unicas_host_es_superhost, categorias_unicas_reservacion_instantanea, columnas_objetivo = cargar_datos()

st.set_page_config(
    page_title="LISBOA",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.markdown("""
    <style>
        .stPlotlyChart {
            width: 100% !important; 
        }
    </style>
""", unsafe_allow_html=True)


imagenB= Image.open("lisboaB.png")
st.sidebar.write(imagenB)
Vista= st.sidebar.selectbox(label="Opciones",options=["Inicio","Datos del DataFrame",'Regresión','Mapa de calor'])
verdeL=["#34a44e"]
verde="#34a44e"    
rojo="#e75f50"
rojoL=["#e75f50"]
azul="#1fb4e7"
azulL=["#1fb4e7"]
if Vista=="Inicio":
    st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:60px'>Airbnb Lisboa</h1>", unsafe_allow_html=True)

    conteo_room_type = df["room_type"].value_counts()
    figuraHOST= px.pie(names=conteo_room_type.index, values=conteo_room_type.values, 
    title=str("Cantidad de hosts por tipo de habitación"), width=1400, height=500,  color_discrete_sequence=colores)
    figuraHOST.update_layout(title_x=0,title_font=dict(color=rojo, size=16))
    figuraREV = px.scatter(data_frame=df, x=df["first_review"], y=df["reviews_per_month"], title=str("Reseñas por mes vs Primera reseña "), color_discrete_sequence=colores)
    figuraREV.update_layout(title_x=0,title_font=dict(color=rojo, size=16))
    figuraREV.update_xaxes(automargin=True)
    figuraREV.update_yaxes(automargin=True)
    figuraPRICE = px.scatter(data_frame=df, x=df["first_review"], y=df["price"], title=str("Precio con Primera reseña "), color_discrete_sequence=colores)
    figuraPRICE.update_layout(title_x=0,title_font=dict(color=rojo, size=16))
    figuraPRICE.update_xaxes(automargin=True)
    figuraPRICE.update_yaxes(automargin=True)

    col1, col2 = st.columns(2)
    col3,col5=st.columns(2)
    with col1: 
        st.plotly_chart(figuraHOST, use_container_width=True)

    with col2:
        st.plotly_chart(figuraREV, use_container_width=True)
    
    with col3:
        st.plotly_chart(figuraPRICE, use_container_widht=True)
    
    df["first_review"] = pd.to_datetime(df["first_review"], errors='coerce')
    df["mes_estreno"] = df["first_review"].dt.to_period("M").astype(str)
    promedios = df.groupby("mes_estreno")["price"].mean().reset_index()
    figura = px.line(data_frame=promedios, x="mes_estreno", y="price", title="Precio promedio por mes de estreno", color_discrete_sequence=verdeL)
    figura.update_layout(title_font_color=rojo)
    st.plotly_chart(figura, use_container_widht=True)

    df["first_review"] = pd.to_datetime(df["first_review"], errors='coerce')
    conteo = df.groupby("mes_estreno").size().reset_index(name="total_alojamientos")
    figuraAloj = px.line(data_frame=conteo, x="mes_estreno", y="total_alojamientos",title="Cantidad de alojamientos estrenados por mes", color_discrete_sequence=rojoL)
    figuraAloj.update_layout(title_font_color=rojo)
    with col5:
        st.plotly_chart(figuraAloj, use_container_widht=True)
    
    st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:50px'>Lisboa: La ciudad de las siete colinas</h1>", unsafe_allow_html=True)

    col10,col11=st.columns(2)
    imagen = Image.open("muelle2.jpg")
    imagen2= Image.open("casas.jpg")
    with col10:
        st.image(imagen, caption="Belém", use_container_width=True)
    with col11:
        st.image(imagen2, caption="Barrio de Alfama", use_container_width=True)


    c1,c2,c3,c4=st.columns(4)
    with c1:
        with st.container():
            st.markdown("""
            <div style="background-color: #f0f8ff; border: 3px solid #50e261; padding: 20px; border-radius: 12px; height:225px">
                <h3 style="color: #2b6e37; text-align:center">Alojamientos con casa o departamento entero</h3>
                <h4 style="text-align: center">17765</h4>
            </div>
            """, unsafe_allow_html=True)
    with c2:
        with st.container():
            st.markdown("""
            <div style="background-color: #f0f8ff; border: 3px solid #50e261; padding: 20px; border-radius: 12px; height:225px">
                <h3 style="color: #2b6e37; text-align:center">Alojamientos con habitaciones privadas</h3>
                <h4 style="text-align: center">5937</h4>
            </div>
            """, unsafe_allow_html=True)
    with c3:
        with st.container():
            st.markdown("""
            <div style="background-color: #f0f8ff; border: 3px solid #50e261; padding: 20px; border-radius: 12px;height:225px">
                <h3 style="color: #2b6e37; text-align:center">Alojamientos con habitaciones de hotel</h3>
                <h4 style="text-align: center">219</h4>
            </div>
            """, unsafe_allow_html=True)
    with c4:
        with st.container():
            st.markdown("""
            <div style="background-color: #f0f8ff; border: 3px solid #50e261; padding: 20px; border-radius: 12px;height:225px">
                <h3 style="color: #2b6e37; text-align:center">Alojamientos con habitaciones compartidas</h3>
                <h4 style="text-align: center">283</h4>
            </div>
            """, unsafe_allow_html=True)

elif Vista == "Datos del DataFrame":
    st.subheader("Columnas del DataFrame")
    st.dataframe(pd.DataFrame({"Nombres de las columnas": df.columns}))
    st.subheader("Columnas de tipo numérico del Dataframe")
    st.dataframe(pd.DataFrame({"Nombres de las columnas": dataframe_numerico.columns}))
    st.subheader("Columnas  de tipo textual del Dataframe")
    st.dataframe(pd.DataFrame({"Nombres de las columnas": dataframe_texto.columns}))
    st.subheader("Datos del DataFrame")
    st.write(df)
    st.subheader('Estadísticos del DataFrame')
    st.write(dataframe_numerico.describe()) 

elif Vista =="Regresión":
    regresiones = st.sidebar.radio(label="Elige el tipo de regresión que se usará.", options=["Regresión lineal simple","Regresión lineal múltiple", "Regresión logística"])
    if regresiones == "Regresión lineal simple":
        st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:60px'>Regresión lineal simple</h1>", unsafe_allow_html=True)
        x_col = st.sidebar.selectbox("Variable independiente", options=columnas_numericas)
        y_col = st.sidebar.selectbox("Variable dependiente", options=columnas_numericas)

        if x_col != y_col:
            df_filtrado = df[[x_col, y_col]].dropna()
            X = df_filtrado[[x_col]]  
            y = df_filtrado[y_col]   

            modelo = LinearRegression()
            modelo.fit(X, y)
            coef_Deter=modelo.score(X=X, y=y)
            coef_Correl=np.sqrt(coef_Deter)
            df_filtrado['prediccion'] = modelo.predict(X)

            fig = px.scatter(df_filtrado, x=x_col, y=y_col,  color_discrete_sequence= colores)
            linea_regresion= st.sidebar.checkbox(label="Mostrar Linea de regresión")
            coeficientes=st.sidebar.checkbox(label="Mostrar coeficientes")
            if linea_regresion:
                fig.add_scatter(x=df_filtrado[x_col], y=df_filtrado['prediccion'], mode='lines', name='Línea de regresión', line=dict(color='#1fb4e7', width=6))
            st.plotly_chart(fig)
            if coeficientes:
                st.markdown(f"<h4 style='text-align: center; color: #2b6e37'; font-size:30px>Coeficiente de determinación (R2): {coef_Deter:.4f}</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='text-align: center; color: #2b6e37'; font-size:30px>Coeficiente de correlación (R): {coef_Correl:.4f}</h4>",unsafe_allow_html=True)

        else:
            st.warning("Selecciona columnas distintas para X y Y.")

    elif regresiones == "Regresión lineal múltiple":

        st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:60px'>Regresión lineal múltiple</h1>", unsafe_allow_html=True)
        Var_Dep2 = st.sidebar.selectbox("Variable dependiente", options=columnas_numericas)
        Vars_Indep2 = st.sidebar.multiselect("Variables independientes", placeholder="Selecciona las variables independientes" ,options=[col for col in columnas_numericas if col != Var_Dep2])
        boton= st.sidebar.button("Mostrar gráfica comparativa")
        
        if boton and Vars_Indep2 and Var_Dep2 not in Vars_Indep2:
            df_filtrado = df[Vars_Indep2 + [Var_Dep2]].dropna()
            X = df_filtrado[Vars_Indep2]
            y = df_filtrado[Var_Dep2]
            modelo2= LinearRegression()
            modelo2.fit(X=X, y=y)
            df_combinado = df_filtrado.melt(id_vars=[Var_Dep2], value_vars=Vars_Indep2, var_name="Variable Independiente", value_name="Comparativas")
            fig = px.scatter(df_combinado, x="Comparativas", y=Var_Dep2, color="Variable Independiente", color_discrete_sequence=colores2)
            st.plotly_chart(fig)
            coef_Deter2=modelo2.score(X=X, y=y)
            coef_Correl2=np.sqrt(coef_Deter2)
            st.markdown(f"<h4 style='text-align: center; color: green'; font-size:80px>Coeficiente de determinación (R2): {coef_Deter2:.4f}</h4>",unsafe_allow_html=True)
            st.markdown(f"<h4 style='text-align: center; color: green'; font-size:30px>Coeficiente de correlación (R): {coef_Correl2:.4f}</h4>",unsafe_allow_html=True)

        else:
            st.warning("Selecciona al menos una variable independiente distinta a la dependiente.")

    elif regresiones == "Regresión logística":
        st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:60px'>Regresión logística</h1>", unsafe_allow_html=True)
        Var_Dep3 = st.sidebar.selectbox("Variable dependiente", options=columnas_logistica)
        Vars_Indep3 = st.sidebar.multiselect("Variables independientes", placeholder="Selecciona las variables independientes" ,options=[col for col in columnas_numericas if col != Var_Dep3])
        if len(Vars_Indep3) > 0 and Var_Dep3:
            df_filtrado3 = df[Vars_Indep3 + [Var_Dep3]].dropna()

            df_filtrado3['neighbourhood_group_cleansed'] = df['neighbourhood_group_cleansed'].replace(
                ["Alenquer", 'Amadora', 'Arruda Dos Vinhos', 'Azambuja', 'Cadaval', 'Cascais', 'Loures', 'Lourinh', 'Mafra', 
                'Odivelas', 'Oeiras', 'Sintra', 'Sobral De Monte Agrao', 'Torres Vedras', 'Vila Franca De Xira'], "No Lisboa"
            )
            df_filtrado3['room_type'] = df['room_type'].replace(
                ['Hotel room', 'Private room', 'Shared room'], "No Entire home"
            )
            

            Min = df['reviews_per_month'].min()
            Max = df['reviews_per_month'].max()
            intervalos = [Min - 1, 1, Max]
            categorias = ["1 o menos", "Más de 1"]
            df_filtrado3['reviews_per_month'] = pd.cut(df['reviews_per_month'], bins=intervalos, labels=categorias, right=True)


            Min = df['review_scores_rating'].min()
            Max = df['review_scores_rating'].max()
            intervalos = [Min - 1, 4.8, Max]
            categorias = ["Menos de 4.8", "4.8 o más"]
            df_filtrado3['review_scores_rating'] = pd.cut(df['review_scores_rating'], bins=intervalos, labels=categorias, right=True)

            X = df_filtrado3[Vars_Indep3]
            y = df_filtrado3[Var_Dep3]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
            
            escalar = StandardScaler()
            X_train = escalar.fit_transform(X_train)
            X_test = escalar.transform(X_test)
            
            algoritmo = LogisticRegression()
            algoritmo.fit(X_train, y_train)
            y_pred = algoritmo.predict(X_test)
            
            matriz = confusion_matrix(y_test, y_pred)
            confirmar=st.sidebar.checkbox("Mostrar matriz de Confusión y métricas")
            if confirmar:
                st.write(matriz)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='binary', pos_label=y.unique()[0])
                precision2 = precision_score(y_test, y_pred, average='binary', pos_label=y.unique()[1])
                recall = recall_score(y_test, y_pred, average='binary', pos_label=y.unique()[0])
                recall2 = recall_score(y_test, y_pred, average='binary', pos_label=y.unique()[1])
                st.markdown("<h3 style='color: green'; font-size:30px>Métricas del modelo</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='color: green'; font-size:30px>Exactitud (accuracy): {accuracy:.4f}</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='color: green'; font-size:30px>Precision (precision T o >): {precision:.4f}</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='color: green'; font-size:30px>Precision (precision F o <): {precision2:.4f}</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='color: green'; font-size:30px>Sensibilidad (sensibilidad T o >): {recall:.4f}</h4>",unsafe_allow_html=True)
                st.markdown(f"<h4 style='color: green'; font-size:30px>Sensibilidad (sensibilidad F o <): {recall2:.4f}</h4>",unsafe_allow_html=True)
        else:
            st.warning("Selecciona al menos una variable independiente.")

elif Vista == "Mapa de calor":
    st.markdown("<h1 style='text-align: center; color: #2b6e37; font-size:60px'>Mapa de calor de correlación</h1>", unsafe_allow_html=True)
    
    var = st.sidebar.multiselect(label="Variables numéricas", options=columnas_numericas)
    if len(var) >= 2:
        mapa = dataframe_numerico[var].corr()
        mapa=mapa.abs()
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(mapa, annot=True, fmt=".2f", cmap='Greens', square=True, cbar=True)
        st.pyplot(fig)
    else:
        st.warning("Selecciona las variables numéricas que deseas correlacionar.")

Overwriting lis.py
